# Simulacion de inventarios

In [203]:
import numpy as np
import matplotlib.pyplot as plt

### Distribucion de probabilidad de la demanda y generacion de variable aleatoria

In [204]:
def generar_var_aleatoria(soporte, distribucion, n):
    dist_acumulada = np.insert(np.cumsum(distribucion), 0, 0)

    if n == 1:
        variable_aleatoria = 1
        u = np.random.uniform(0, 1)
        for j in range(len(dist_acumulada)-1):
            if dist_acumulada[j] <= u < dist_acumulada[j+1]:
                variable_aleatoria = soporte[j]
                break
        return variable_aleatoria
    else:
        variables_aleatorias = np.zeros(n, dtype=int)
        for i in range(n):
            u = np.random.uniform(0,1)
            for j in range(len(dist_acumulada)-1):
                if dist_acumulada[j] <= u < dist_acumulada[j+1]:
                    variables_aleatorias[i] = soporte[j]
                    break
    return variables_aleatorias
    

In [205]:
soporte_demanda = [
    35, 36, 37, 38, 39, 40, 41, 42, 43, 44,
    45, 46, 47, 48, 49, 50, 51, 52, 53, 54,
    55, 56, 57, 58, 59, 60
]

distribucion_demanda = [
    0.010, 0.015, 0.020, 0.020, 0.022, 0.023, 0.025, 0.027, 0.028, 0.029,
    0.035, 0.045, 0.060, 0.065, 0.070, 0.080, 0.075, 0.070, 0.065, 0.060,
    0.050, 0.040, 0.030, 0.016, 0.015, 0.005
]

def generar_demanda(n = 1):
    return generar_var_aleatoria(soporte_demanda, distribucion_demanda, n)

print(generar_demanda(10))

[47 48 55 43 50 50 40 50 49 52]


In [206]:
soporte_tiempos = [1,2,3]

distribucion_tiempos = [0.3, 0.4, 0.3]

def generar_tiempo(n = 1):
    return generar_var_aleatoria(soporte_tiempos, distribucion_tiempos, n)

print(generar_tiempo(10))

[3 2 3 2 2 2 3 2 2 1]


In [ ]:
factores_estacionales = np.array([
    1.20, 1.00, 0.90, 0.80, 0.80, 0.70,
    0.80, 0.90, 1.00, 1.20, 1.30, 1.40
])

demanda_control = [
    64, 52, 47, 31, 
    40, 25, 34, 38,
    48, 58, 69, 70
]

tiempos_control = [
    1,0,0,0,
    0,3,0,0,
    0,0,3,0
]

def simulacion(inventario_inicial = 150, orden = 200, reorden = 100):
    faltante_total = 0
    n_pedidos = 0
    inventario_promedio = 0

    inventario = inventario_inicial
    pedido_activo = False
    tiempo_espera = 0
    faltante = 0
    promedio = 0

    for i in range(12):
        if pedido_activo and tiempo_espera == 0:
            inventario += orden
            pedido_activo = False
            tiempo_espera = 0
        elif pedido_activo and tiempo_espera > 0:
            tiempo_espera -= 1

        if faltante > 0:
            inventario -= faltante
            faltante = 0

        #print(f"Mes {i+1}:\nInventario inicial: {inventario:.2f}")

        demanda = generar_demanda(1) * factores_estacionales[i]
        #demanda = demanda_control[i]
        inventario = inventario - demanda

        if not pedido_activo and inventario < reorden:
            pedido_activo = True
            tiempo_espera = generar_tiempo(1)
            #tiempo_espera = tiempos_control[i]
            n_pedidos += 1

        if inventario < 0:
            faltante = abs(inventario)
            inventario = 0
            promedio = (demanda - faltante)**2 / (2*demanda)
        else:
            promedio = (demanda + 2 * inventario) / 2

        inventario_promedio += promedio
        faltante_total += faltante

        #print(f"Demanda: {demanda:.2f}, Inventario final: {inventario:.2f}")
        #print(f"Faltante: {faltante:.2f}, Inventario promedio: {promedio:.2f}\n")
        #print(f"Pedidos: {n_pedidos}, Tiempo de espera: {tiempo_espera}") if inventario < reorden else print("No hay pedidos activos")
    
    return n_pedidos, faltante_total, inventario_promedio

    #print(f"faltante total: {faltante_total}, inventario promedio: {inventario_promedio:.2f}\n")
    #print(f"\nCosto de ordenar: {n_pedidos * 100}\nCosto de faltantes: {faltante_total * 50}\nCosto de inventario: {inventario_promedio * 1.67}\n")

    #print(f"Costo total: {n_pedidos * 100 + faltante_total * 50 + inventario_promedio * 1.67}")

In [723]:
simulacion()

Mes 1:
Inventario inicial: 150.00
Demanda: 43.20, Inventario final: 106.80
Faltante: 0.00, Inventario promedio: 128.40

No hay pedidos activos
Mes 2:
Inventario inicial: 106.80
Demanda: 58.00, Inventario final: 48.80
Faltante: 0.00, Inventario promedio: 77.80

Pedidos: 1, Tiempo de espera: 2
Mes 3:
Inventario inicial: 48.80
Demanda: 46.80, Inventario final: 2.00
Faltante: 0.00, Inventario promedio: 25.40

Pedidos: 1, Tiempo de espera: 1
Mes 4:
Inventario inicial: 2.00
Demanda: 39.20, Inventario final: 0.00
Faltante: 37.20, Inventario promedio: 0.05

Pedidos: 1, Tiempo de espera: 0
Mes 5:
Inventario inicial: 162.80
Demanda: 41.60, Inventario final: 121.20
Faltante: 0.00, Inventario promedio: 142.00

No hay pedidos activos
Mes 6:
Inventario inicial: 121.20
Demanda: 37.10, Inventario final: 84.10
Faltante: 0.00, Inventario promedio: 102.65

Pedidos: 2, Tiempo de espera: 1
Mes 7:
Inventario inicial: 84.10
Demanda: 43.20, Inventario final: 40.90
Faltante: 0.00, Inventario promedio: 62.50

P

In [330]:
demanda = generar_demanda(12)
print(demanda)
print(demanda * factores_estacionales)
print(sum(factores_estacionales))

[49 53 57 52 59 47 42 56 50 54 41 51]
[58.8 53.  51.3 41.6 47.2 32.9 33.6 50.4 50.  64.8 53.3 71.4]
12.000000000000002


In [331]:
234*True

234